In [ ]:
from vpython import *
import math
import random

In [ ]:
#"b":{"id":"b","x":324,"y":530,"z":342,"lock":False,"label":"Banana"}
node_list ={
    "0":{"id":"0","x": random.randint(1,501),"y":random.randint(1,501),"z":random.randint(1,501),"lock":False},
    "1":{"id":"1","x": random.randint(1,501),"y":random.randint(1,501),"z":random.randint(1,501),"lock":False},
    "2":{"id":"2","x": random.randint(1,501),"y":random.randint(1,501),"z":random.randint(1,501),"lock":False},
    "3":{"id":"3","x": random.randint(1,501),"y":random.randint(1,501),"z":random.randint(1,501),"lock":False},
    "4":{"id":"4","x": random.randint(1,501),"y":random.randint(1,501),"z":random.randint(1,501),"lock":False},
}

edge_list = [
    {"source":"0", "target":"1"}, 
    {"source":"0", "target":"2"}, 
    {"source":"0", "target":"3"},
    {"source":"0", "target":"4"},
    {"source":"1", "target":"2"},
    {"source":"1", "target":"3"},
    {"source":"1", "target":"4"},
    {"source":"2", "target":"3"},
    {"source":"2", "target":"4"},
    {"source":"3", "target":"4"}
]

natLength = 200
ks = 0.3
kg = 90
maxRepulsion = 300
maxZ = 500
ZDecreaser = 1
loopInterval = 0.01
maxLoops = 1000


#for key, value in d.iteritems():
def draw_nodes(list):
    for id, properties in list.iteritems():
        xcoords = properties["x"]
        ycoords = properties["y"]
        zcoords = properties["z"]
        object = sphere(pos=vector(xcoords,ycoords,zcoords),
                        radius=10,
                        color=color.green,
                        make_trail = True)
        node_id = properties["id"]
        properties["sphere"] = object
    draw_label(list)
def draw_edge(list):
    for edge in list:
        for id,properties in node_list.iteritems():
            if edge["source"] == id:
                sourceX = properties["x"]
                sourceY = properties["y"]
                sourceZ = properties["z"]
            if edge["target"] == id:
                targetX = properties["x"]
                targetY = properties["y"]
                targetZ = properties["z"]     
        object = arrow(pos = vector(sourceX, sourceY, sourceZ),
                       axis = vector(targetX-sourceX, targetY-sourceY, targetZ-sourceZ), 
                       shaftwidth = 5)
        edge["arrow"] = object               

def draw_label(list):
    for id,properties in list.iteritems():
        xcoords = properties["x"]
        ycoords = properties["y"]
        zcoords = properties["z"]
        if properties.get("label"):
            textMess = properties["label"]
            object = label( pos=vec(xcoords, ycoords, zcoords), text= textMess)
            node_id = id
            properties["sign"] = object
            
def draw(nodes, edges):
    draw_edge(edges)
    draw_nodes(nodes)


In [ ]:
def move(forceX,forceY,forceZ,node_id,maxZ,startedIter):
    maxed = maxZ
    node = node_list[node_id]
    xPos = node["x"]
    yPos = node["y"]
    zPos = node["z"]
    newxPos = xPos+forceX
    newyPos = yPos+forceY
    newzPos = zPos+forceZ
    if startedIter:
        if newzPos > maxed:
            newzPos = maxed
        if newzPos < -maxed:
            newzPos = -maxed
    node["sphere"].pos=vector(newxPos,newyPos,newzPos)
    if node.get("label"):
        node["sign"].pos=vector(newxPos,newyPos,newzPos)
    node["x"] = newxPos
    node["y"] = newyPos
    node["z"] = newzPos


def move_edges():
    for edge in edge_list:
        sourceNode = node_list[edge["source"]]
        targetNode = node_list[edge["target"]]
        arrow = edge["arrow"]
        sourcex = sourceNode["x"]
        sourcey = sourceNode["y"]
        sourcez = sourceNode["z"]
        targetx = targetNode["x"] - sourceNode["x"]
        targety = targetNode["y"] - sourceNode["y"]
        targetz = targetNode["z"] - sourceNode["z"]
        arrow.pos = vector(sourcex,sourcey,sourcez) 
        arrow.axis = vector(targetx, targety, targetz)

In [ ]:
def hookesLaw(a,b):
    xdist = b["x"]-a["x"]
    ydist = b["y"]-a["y"]
    zdist = b["z"]-a["z"]
    d3 = xdist*xdist+ydist*ydist+zdist*zdist
    d3 = math.sqrt(d3)
    distance = d3-natLength
    if d3 == 0:
        return [0,0,0]
    if d3 != 0:
        force = ks*distance/d3
    return [xdist*force, ydist*force,zdist*force ]

def coulombsLaw(a,b):
    xdist = b["x"]-a["x"]
    ydist = b["y"]-a["y"]
    zdist = b["z"]-a["z"]
    d3 = xdist*xdist+ydist*ydist+zdist*zdist
    d3 = math.sqrt(d3)
    if d3 == 0 or d3 > maxRepulsion:
        return [0,0,0]
    if d3 != 0 and d3 < maxRepulsion:
        force = kg/(d3*d3)
    return [-force*xdist,-force*ydist,-force*zdist]


In [ ]:
def loop():
    draw(node_list,edge_list)
    numLoops = 0
    startedIter = False
    maxed = maxZ
    while True:
        sleep(loopInterval)
        if numLoops > 240:
            startedIter = True
        for node1, properties1 in node_list.iteritems():
            netForceX = 0
            netForceY = 0
            netForceZ = 0
            if properties1["lock"] == False:
                for node2, properties2 in node_list.iteritems():
                    if node1 != node2:
                        addedXCForce = coulombsLaw(properties1,properties2)[0]
                        addedYCForce = coulombsLaw(properties1,properties2)[1]
                        addedZCForce = coulombsLaw(properties1,properties2)[2]
                        netForceX = netForceX + addedXCForce
                        netForceY = netForceY + addedYCForce
                        netForceZ = netForceZ + addedZCForce
                for edge in edge_list:
                    if node1 == edge["source"]:
                        for search,prop in node_list.iteritems():
                            if edge["target"] == search:   
                                addedXHForce = hookesLaw(properties1, prop)[0]
                                addedYHForce = hookesLaw(properties1, prop)[1]
                                addedZHForce = hookesLaw(properties1, prop)[2]
                                netForceX = netForceX + addedXHForce
                                netForceY = netForceY + addedYHForce
                                netForceZ = netForceZ + addedZHForce
                    if node1 == edge["target"]:
                        for search,prop in node_list.iteritems():
                            if edge["source"] == search:  
                                addedXHForce = hookesLaw(properties1, prop)[0]
                                addedYHForce = hookesLaw(properties1, prop)[1]
                                addedZHForce = hookesLaw(properties1, prop)[2]
                                netForceX = netForceX + addedXHForce
                                netForceY = netForceY + addedYHForce
                                netForceZ = netForceZ + addedZHForce

            move(netForceX,netForceY,netForceZ,node1, maxed,startedIter)
        if startedIter and maxed !=0:
            maxed = maxed - ZDecreaser
        if maxLoops > 0 and numLoops > maxLoops:
            break


                
        move_edges()
        numLoops = numLoops + 1

In [ ]:
loop()